# Fit the Z peak in CMS dimuon data from 2012

We use the RooFit dataset from the 2012 CMS open data that we created in the other notebook. We create a binned dataset in the range 70 - 110 GeV, and fit the peak using a Bukin PDF, a Gaussian-like distribution that allows for asymmetric tails.

The full dimuon spectrum looks like this:
![spectrum.png](spectrum.png)

**Author**: Stephan Hageboeck (CERN)

## 1. Open the file and prepare the binned dataset
We convert the RooDataSet of the entire dimuon spectrum to a binned dataset in the range [70, 110] GeV

In [1]:
TFile file2("RFDataset.root", "READ");
auto dataset = file2.Get<RooDataSet>("dataset");
dataset->Print();

RooRealVar x("x", "m_{#mu#mu}", 70, 110, "GeV");
RooDataHist hist("hist", "hist", {x});
hist.add(*dataset, "x>70 && x<110");

file2.Close();


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

RooDataSet::dataset[x] = 25794885 entries


## 2. Declare background and signal fit models
For the background distribution, we use a sum of the three 2nd-order [Bernstein polynomials](https://root.cern.ch/doc/master/classRooBernstein.html). These are useful for describing somewhat (but not really) flat background distributions.

For the signal model, we use the [Bukin PDF](https://root.cern.ch/doc/master/classRooBukinPdf.html), which is good for fitting asymmetric peaks.

In [2]:
RooRealVar a("a", "Bernstein 1st coefficient", 0, 100);
RooRealVar b("b", "Bernstein 2nd coefficient", 0, 100);
RooRealVar c("c", "Bernstein 3rd coefficient", 0, 100);
RooBernstein bernstein("bernstein", "Bernstein polynomials", x, {a, b, c});

RooRealVar mean("mean", "mean", 80, 100, "GeV");
RooRealVar sigma("sigma", "sigma", 1, 10, "GeV");
RooRealVar xi("asymmetry", "Peak asymmetry", 0, -0.9, 0.9); 
RooRealVar tailL("tailL", "Left tail", 0, -0.9999, -0.00001);
RooRealVar tailR("tailR", "Right tail", 0, 0.00001, 0.9999);
RooBukinPdf buk("buk", "Bukin distribution",
                x, mean, sigma,
                xi, tailL, tailR);

## 3. Combine the signal and background models
We define parameters for counting the number of signal and background events. We expect a few million events for each. To help the minimiser, we should keep these parameters close to 1. We can achieve this by scaling them up by a million using a RooFormulaVar.

In [3]:
RooRealVar sig("sig", "sig", 0., 10., "M");
RooRealVar bkg("bkg", "bkg", 0., 10., "M");
RooFormulaVar sigM("sigM", "1.E6 * sig", sig);
RooFormulaVar bkgM("bgkM", "1.E6 * bkg", bkg);

RooAddPdf sumModel("sum", "Bukin + Bernstein", {buk, bernstein}, {sigM, bkgM});

### Visualising the model
RooFit can export the model in graphViz format.

In [4]:
sumModel.graphVizTree("sumModel.dot");

Executables such as `dot` or `neato` can be used to layout the graph.

In [5]:
%shell dot -Tpng -o sumModel.png sumModel.dot

The original model is:
<img src='sumModel.png' width="1024">

## 4. Fit to data
By passing the dataset to the fit model, we can minimise the likelihood. We also ask RooFit to save the fit result for later inspection.

In [6]:
auto result = sumModel.fitTo(hist, RooFit::Save());

[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:NumericIntegration -- RooRealIntegral::init(buk_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:NumericIntegration -- RooRealIntegral::init(buk_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (buk,bernstein)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a            5.00000e+01  1.00000e+01    0.00000e+00  1.00000e+02
     2 asymmetry    0.00000e+00  1.80000e-01   -9.00000e-01  9.00000e-01
     3 b            5.00000e+01  1.00000e+01    0.00000e+00  1.00000e+02
     4 bkg          5.00000e+0

## 5. Inspect the fit result
We have a Z peak at 90.96 +- 0.01 GeV, so CMS's open-data muons are probably not fully calibrated. We see 4.4 million Z bosons and 1.4 million background events.

In [7]:
result->Print();


  RooFitResult: minimized FCN value: -6.79643e+07, estimated distance to minimum: 0.29016
                covariance matrix quality: Full matrix, but forced positive-definite
                Status : MINIMIZE=-1 HESSE=4 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                     a    6.0855e+00 +/-  3.04e-01
             asymmetry   -7.5427e-02 +/-  6.45e-04
                     b    1.0154e+01 +/-  5.07e-01
                   bkg    1.4493e+00 +/-  2.90e-03
                     c    8.4250e-01 +/-  4.29e-02
                  mean    9.0962e+01 +/-  2.37e-03
                   sig    4.4055e+00 +/-  3.37e-03
                 sigma    2.1346e+00 +/-  1.70e-03
                 tailL   -1.0039e-05 +/-  1.02e-05
                 tailR    4.4270e-02 +/-  1.87e-03



## 6. Plot the result
We use javascript root to create a plot that we can interact with in this notebook (e.g. zoom or drag things around).
We also plot our fit model multiple times:
- The full model
- The signal component
- The background component

We add some of the post-fit parameters to the plot. The `Format("NEU")` stands for:
**N**ame  **E**rror  **U**nit
We give each curve we plot a name, so we can use it to build a legend.

One can see in the plot that the Bukin distribution cannot fully capture what really happens in CMS, but this is good enough for a first estimate.

In [8]:
%jsroot on
auto c1 = new TCanvas("c1", "RF CMS Z Peak", 1024, 768);
auto frame = x.frame();

hist.plotOn(frame, RooFit::Name("data"));
sumModel.plotOn(frame, RooFit::Name("full"), RooFit::LineWidth(4));
sumModel.plotOn(frame, RooFit::Name("bkg"), RooFit::Components("bernstein"), RooFit::LineColor(kGreen), RooFit::LineWidth(4));
sumModel.plotOn(frame, RooFit::Name("sig"), RooFit::Components("buk"), RooFit::LineColor(kRed), RooFit::LineWidth(4));
sumModel.paramOn(frame,
                 RooFit::Label("CMS open data dimuon Z peak"),
                 RooFit::Parameters(RooArgSet{mean, sigma, xi, tailL, tailR, sig, bkg}),
                 RooFit::Format("NEU", RooFit::AutoPrecision(1)) );

frame->Draw();

TLegend leg(0.15, 0.65, 0.45, 0.9);
leg.AddEntry(frame->getCurve("data"), "CMS Open Data", "P");
leg.AddEntry(frame->getCurve("full"), "Full model", "L");
leg.AddEntry(frame->getCurve("sig"), "Bukin", "L");
leg.AddEntry(frame->getCurve("bkg"), "Bernstein", "L");
leg.SetBorderSize(0);
leg.SetFillStyle(0);
leg.Draw();

c1->Draw();

[#1] INFO:NumericIntegration -- RooRealIntegral::init(buk_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(sum) directly selected PDF components: (bernstein)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(sum) indirectly selected PDF components: ()
[#1] INFO:NumericIntegration -- RooRealIntegral::init(buk_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(sum) directly selected PDF components: (buk)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(sum) indirectly selected PDF components: ()
[#1] INFO:NumericIntegration -- RooRealIntegral::init(buk_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
